# HW 2-2 Add more statistics to analyze the an ONNX model

## 2-2-1. model characteristics

- operator types
    - Print the number of onnx operators, list out unique onnx operator names
- For each operator
    - print out the attributes ranges
    - e.g. For a Conv2D layer, print its width, height, channel, dialation, stride, and kernel size

In [172]:
import onnx
onnx_model = onnx.load('./mobilenetv2-10.onnx')

# The model is represented as a protobuf structure and it can be accessed
# using the standard python-for-protobuf methods

## list all the operator types in the model
node_list = []
# count = []
for i in onnx_model.graph.node:
    if (i.op_type not in node_list):
        node_list.append(i.op_type)
        # count.append(1)
    else:
        idx = node_list.index(i.op_type)
        # count[idx] = count[idx]+1
print(node_list)
# print(count)

['Conv', 'Clip', 'Add', 'GlobalAveragePool', 'Shape', 'Constant', 'Gather', 'Unsqueeze', 'Concat', 'Reshape', 'Gemm']


In [35]:
def get_size(shape):
    dims = []
    size = 1
    # print((shape))
    for dim in shape.dim:
        if dim.dim_value > 0:
            dims.append(dim.dim_value)
            size *= dim.dim_value
        elif dim.dim_param:
            dims.append(1)  # 保存符號名稱
            size = 1  # 表示無法計算完整大小
        else:
            dims.append(-1)  # 未定義維度
            size = 0
    return dims, size
def get_size2(shape):
    dims = shape.dims
    size = 1
    for dim in dims:
        size *= dim
    return dims, size

In [203]:
import onnx

onnx_model = onnx.load('./mobilenetv2-10.onnx')
onnx_model = onnx.shape_inference.infer_shapes(onnx_model)

# 提取名稱字典（提高查找效率）
input_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.input)}
initializer_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.initializer)}
value_info_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.value_info)}


# 遍歷所有節點，提取每個 operator 的屬性
for node in onnx_model.graph.node:
    print(f"\n-- Operator '{node.op_type}' (name: '{node.name}') --")
    
    # 打印輸入資訊
    # print(node)
    for input_name in node.input:
        if input_name in input_dict:
            idx = input_dict[input_name]
            dims, size = get_size(onnx_model.graph.input[idx].type.tensor_type.shape)
            # print(f"Input {input_name} (data) has {size} elements, dims = {dims}")
            print(f"{input_name} : {dims}")
        elif input_name in initializer_dict:
            idx = initializer_dict[input_name]
            dims, size = get_size2(onnx_model.graph.initializer[idx])
            # print(f"Input {input_name} (weight/bias) has {size} elements, dims = {dims}")
            print(f"{input_name} : {dims}")
        elif input_name in value_info_dict:
            idx = value_info_dict[input_name]
            dims, size = get_size(onnx_model.graph.value_info[idx].type.tensor_type.shape)
            # print(f"Input {input_name} (intermediate) has {size} elements, dims = {dims}")
            print(f"{input_name} : {dims}")
        else:
            print(f"Input {input_name} not found in input, initializer, or value_info")
    
    # 打印屬性值
    if node.attribute:
        print("Attributes:")
        for attr in node.attribute:
            if attr.type == onnx.AttributeProto.INT:
                print(f"  {attr.name}: {attr.i}")
            elif attr.type == onnx.AttributeProto.INTS:
                print(f"  {attr.name}: {list(attr.ints)}")
            elif attr.type == onnx.AttributeProto.FLOAT:
                print(f"  {attr.name}: {attr.f}")
            elif attr.type == onnx.AttributeProto.STRING:
                # print("print(attr.type)="+attr.type)
                print(f"  {attr.name}: {attr.s.decode('utf-8')}")
            else:
                print(f"  {attr.name}: {attr} (type: {attr.type})")
    else:
        print("  No attributes defined")


-- Operator 'Conv' (name: 'Conv_0') --
input : [1, 3, 224, 224]
475 : [32, 3, 3, 3]
476 : [32]
Attributes:
  dilations: [1, 1]
  group: 1
  kernel_shape: [3, 3]
  pads: [1, 1, 1, 1]
  strides: [2, 2]

-- Operator 'Clip' (name: 'Clip_1') --
474 : [1, 32, 112, 112]
Attributes:
  max: 6.0
  min: 0.0

-- Operator 'Conv' (name: 'Conv_2') --
317 : [1, 32, 112, 112]
478 : [32, 1, 3, 3]
479 : [32]
Attributes:
  dilations: [1, 1]
  group: 32
  kernel_shape: [3, 3]
  pads: [1, 1, 1, 1]
  strides: [1, 1]

-- Operator 'Clip' (name: 'Clip_3') --
477 : [1, 32, 112, 112]
Attributes:
  max: 6.0
  min: 0.0

-- Operator 'Conv' (name: 'Conv_4') --
320 : [1, 32, 112, 112]
481 : [16, 32, 1, 1]
482 : [16]
Attributes:
  dilations: [1, 1]
  group: 1
  kernel_shape: [1, 1]
  pads: [0, 0, 0, 0]
  strides: [1, 1]

-- Operator 'Conv' (name: 'Conv_5') --
480 : [1, 16, 112, 112]
484 : [96, 16, 1, 1]
485 : [96]
Attributes:
  dilations: [1, 1]
  group: 1
  kernel_shape: [1, 1]
  pads: [0, 0, 0, 0]
  strides: [1, 1]


## 2-2-2. Data bandwidth requirement

- Assuming all required data is moved only once, how much data bandwidth is required to do model inference.

In [211]:
onnx_model = onnx.load("mobilenetv2-10.onnx")
onnx_model = onnx.shape_inference.infer_shapes(onnx_model)

# 提取名稱字典
input_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.input)}
initializer_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.initializer)}
value_info_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.value_info)}
output_dict = {k.name: idx for idx, k in enumerate(onnx_model.graph.output)}

# 假設數據類型為 float32（4 字節）
dtype_size = 4  # float32 每元素 4 字節
total_bytes = 0
# 遍歷所有節點，提取每個 operator 的屬性
for node in onnx_model.graph.node:
    # print(f"\n-- Operator '{node.op_type}' (name: '{node.name}') --")
    
    # 打印輸入資訊
    # print(node)
    for input_name in node.input:
        if input_name in input_dict:
            idx = input_dict[input_name]
            dims, size = get_size(onnx_model.graph.input[idx].type.tensor_type.shape)
            print(f"Input {input_name} (data) has {size} elements")
            total_bytes += size *dtype_size
        elif input_name in initializer_dict:
            idx = initializer_dict[input_name]
            dims, size = get_size2(onnx_model.graph.initializer[idx])
            print(f"Input {input_name} (weight/bias) has {size} elements")
            total_bytes += size*dtype_size
        elif input_name in value_info_dict:
            idx = value_info_dict[input_name]
            dims, size = get_size(onnx_model.graph.value_info[idx].type.tensor_type.shape)
            print(f"Input {input_name} (intermediate) has {size} elements")
            total_bytes += size*dtype_size
        elif input_name in output_dict:
            dims, size = get_size(onnx_model.graph.value_info[idx].type.tensor_type.shape)
            print(f"Input {input_name} (intermediate) has {size} elements")
            total_bytes += size*dtype_size
        else:
            print(f"Input {input_name} not found in input, initializer, or value_info")
for output_tensor in onnx_model.graph.output:
    dims, size = get_size(output_tensor.type.tensor_type.shape)
    print(f"Output {output_tensor.name} (intermediate) has {size} elements")
    total_bytes += size*dtype_size

Input input (data) has 150528 elements
Input 475 (weight/bias) has 864 elements
Input 476 (weight/bias) has 32 elements
Input 474 (intermediate) has 401408 elements
Input 317 (intermediate) has 401408 elements
Input 478 (weight/bias) has 288 elements
Input 479 (weight/bias) has 32 elements
Input 477 (intermediate) has 401408 elements
Input 320 (intermediate) has 401408 elements
Input 481 (weight/bias) has 512 elements
Input 482 (weight/bias) has 16 elements
Input 480 (intermediate) has 200704 elements
Input 484 (weight/bias) has 1536 elements
Input 485 (weight/bias) has 96 elements
Input 483 (intermediate) has 1204224 elements
Input 325 (intermediate) has 1204224 elements
Input 487 (weight/bias) has 864 elements
Input 488 (weight/bias) has 96 elements
Input 486 (intermediate) has 301056 elements
Input 328 (intermediate) has 301056 elements
Input 490 (weight/bias) has 2304 elements
Input 491 (weight/bias) has 24 elements
Input 489 (intermediate) has 75264 elements
Input 493 (weight/bias

In [212]:
print(f"{total_bytes/float(1000000)} MB")

67.680108 MB


In [6]:
import onnx
from onnx import shape_inference
from os import path
import sys
from tabulate import tabulate
from onnx import onnx_ml_pb2 as xpb2


onnx_model = onnx.load("./mobilenetv2-10.onnx", load_external_data=False)
onnx.checker.check_model(onnx_model)

inferred_model = shape_inference.infer_shapes(onnx_model)
print('shape inference complete ...')

def _parse_element(elem: xpb2.ValueInfoProto):
    name = getattr(elem, 'name', "None")
    data_type = "NA"
    shape_str = "NA"
    etype = getattr(elem, 'type', False)
    if etype:
        ttype = getattr(etype, 'tensor_type', False)
        if ttype:
            data_type = getattr(ttype, 'elem_type', 0)
            shape = getattr(elem.type.tensor_type, "shape", False)
            if shape:
                shape_str = "["
                dims = getattr(shape, 'dim', [])
                # print(dims)
                for dim in dims:
                    vals = getattr(dim, 'dim_value', "?")
                    # print(vals)
                    if vals == 0: vals=1
                    shape_str += (str(vals) + ",")
                shape_str = shape_str.rstrip(",")
                shape_str += "]"
    return name, data_type, shape_str

def get_valueproto_or_tensorproto_by_name(name: str, graph: xpb2.GraphProto):
    for i, node in enumerate(inferred_model.graph.node):
            if node.name == "":
                inferred_model.graph.node[i].name = str(i)
    input_nlist = [k.name for k in graph.input]
    initializer_nlist = [k.name for k in graph.initializer]
    value_info_nlist = [k.name for k in graph.value_info]
    output_nlist = [k.name for k in graph.output]

    # get tensor data
    if name in input_nlist:
        idx = input_nlist.index(name)
        return graph.input[idx], int(1)
    elif name in value_info_nlist:
        idx = value_info_nlist.index(name)
        return graph.value_info[idx], int(2)
    elif name in initializer_nlist:
        idx = initializer_nlist.index(name)
        return graph.initializer[idx], int(3)
    elif name in output_nlist:
        idx = output_nlist.index(name)
        return graph.output[idx], int(4)
    else:
        print("[ERROR MASSAGE] Can't find the tensor: ", name)
        print('input_nlist:\n', input_nlist)
        print('===================')
        print('value_info_nlist:\n', value_info_nlist)
        print('===================')
        print('initializer_nlist:\n', initializer_nlist)
        print('===================')
        print('output_nlist:\n', output_nlist)
        print('===================')
        return False, 0

def cal_tensor_mem_size(elem_type: str, shape: [int]):
    """ given the element type of the tensor and its shape, and return its memory size.

    Utility.

    Args:
        ttype: the type of the element of the given tensor. format: 'int', ...
        shape: the shape of the given tensor. format: [] of int

    Returns:
        mem_size: int
    """
    # init
    mem_size = int(1)
    # traverse the list to get the number of the elements
    for num in shape:
        mem_size *= num
    # multiple the size of variable with the number of the elements
    # "FLOAT": 1,
    # "UINT8": 2,
    # "INT8": 3,
    # "UINT16": 4,
    # "INT16": 5,
    # "INT32": 6,
    # "INT64": 7,
    # # "STRING" : 8,
    # "BOOL": 9,
    # "FLOAT16": 10,
    # "DOUBLE": 11,
    # "UINT32": 12,
    # "UINT64": 13,
    # "COMPLEX64": 14,
    # "COMPLEX128": 15
    if elem_type == 1:
        mem_size *= 4
    elif elem_type == 2:
        mem_size *= 1
    elif elem_type == 3:
        mem_size *= 1
    elif elem_type == 4:
        mem_size *= 2
    elif elem_type == 5:
        mem_size *= 2
    elif elem_type == 6:
        mem_size *= 4
    elif elem_type == 7:
        mem_size *= 8
    elif elem_type == 9:
        mem_size *= 1
    elif elem_type == 10:
        mem_size *= 2
    elif elem_type == 11:
        mem_size *= 8
    elif elem_type == 12:
        mem_size *= 4
    elif elem_type == 13:
        mem_size *= 8
    elif elem_type == 14:
        mem_size *= 8
    elif elem_type == 15:
        mem_size *= 16
    else:
        print("Undefined data type")

    return mem_size



def get_bandwidth(graph: xpb2.GraphProto):
    try:
        mem_BW_list = []
        total_mem_BW = 0
        unknown_tensor_list = []
        # traverse all the nodes
        for nodeProto in graph.node:
            # init variables
            # print(nodeProto)
            read_mem_BW_each_layer = 0
            write_mem_BW_each_layer = 0
            total_each_layer = 0
            # traverse all input tensor
            for input_name in nodeProto.input:
                # get the TensorProto/ValueInfoProto by searching its name
                proto, type_Num = get_valueproto_or_tensorproto_by_name(
                    input_name, graph)
                # print(proto,type_Num)
                # parse the ValueInfoProto/TensorProto
                if proto:
                    if type_Num == 3:
                        dtype = getattr(proto, 'data_type', False)
                        # get the shape of the tensor
                        shape = getattr(proto, 'dims', [])
                        # print(shape_str)
                        
                    elif type_Num == 1 or type_Num == 2:
                        name, dtype, shape_str = _parse_element(proto)
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        # print(name,shape_str)
                        shape = []
                        # print(f"3 = {dtype}, shape={shape}")
                        for dim in shape_str:
                            if len(dim) > 0:
                                shape.append(int(dim))
                    else:
                        print(
                            '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                            input_name, ' is from a wrong list !')
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        input_name, ' is no found !')
                    unknown_tensor_list.append(
                        (nodeProto.name, input_name, nodeProto.op_type))
                # calculate the tensor size in btye
                print(nodeProto.op_type,shape,cal_tensor_mem_size(dtype, shape))
                read_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)

            # traverse all output tensor
            for output_name in nodeProto.output:
                # get the TensorProto/ValueInfoProto by searching its name
                proto, type_Num = get_valueproto_or_tensorproto_by_name(
                    output_name, graph)
                # parse the ValueInfoProto
                # print(f"type_Num={type_Num}")
                if proto:
                    if type_Num == 2 or type_Num == 4:
                        # name, dtype, shape = utils._parse_ValueInfoProto(proto)
                        name, dtype, shape_str = _parse_element(proto)
                        # print(name,shape_str)
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        shape = []
                        # print(f"{type_Num} = {dtype}, shape={shape}")
                        for dim in shape_str:
                            if len(dim) > 0:
                                shape.append(int(dim))
                    else:
                        print(
                            '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                            output_name, ' is from a wrong list !')
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        input_name, ' is no found !')
                    unknown_tensor_list.append(
                        (nodeProto.name, output_name, nodeProto.op_type))
                # calculate the tensor size in btye
                write_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)

            # cal total bw
            total_each_layer = read_mem_BW_each_layer + write_mem_BW_each_layer

            # store into tuple
            temp_tuple = (nodeProto.name, read_mem_BW_each_layer,
                        write_mem_BW_each_layer, total_each_layer)
            #append it
            mem_BW_list.append(temp_tuple)
            # accmulate the value
            total_mem_BW += total_each_layer

        # display the mem_bw of eahc layer
        columns = ['layer', 'read_bw', 'write_bw', 'total_bw']
        # resort the list
        mem_BW_list = sorted(mem_BW_list,
                             key=lambda Layer: Layer[1],
                             reverse=True)
        print(tabulate(mem_BW_list, headers=columns))
        print(
            '====================================================================================\n'
        )
        # display it
        print(
            "The memory bandwidth for processor to execute a whole model without on-chip-buffer is: \n",
            total_mem_BW, '(bytes)\n',
            float(total_mem_BW) / float(1000000), '(MB)\n')
        # display the unknown tensor
        columns = ['op_name', 'unfound_tensor', 'op_type']
        print(tabulate(unknown_tensor_list, headers=columns))
        print(
            '====================================================================================\n'
        )
    except Exception as e:
        print("[ERROR MASSAGE] Unable to display: " + str(e))
        return False

    return True

#從這裡開始
print("start")
get_bandwidth(inferred_model.graph)


shape inference complete ...
start
Conv [1, 3, 224, 224] 602112
Conv [32, 3, 3, 3] 3456
Conv [32] 128
Clip [1, 32, 112, 112] 1605632
Conv [1, 32, 112, 112] 1605632
Conv [32, 1, 3, 3] 1152
Conv [32] 128
Clip [1, 32, 112, 112] 1605632
Conv [1, 32, 112, 112] 1605632
Conv [16, 32, 1, 1] 2048
Conv [16] 64
Conv [1, 16, 112, 112] 802816
Conv [96, 16, 1, 1] 6144
Conv [96] 384
Clip [1, 96, 112, 112] 4816896
Conv [1, 96, 112, 112] 4816896
Conv [96, 1, 3, 3] 3456
Conv [96] 384
Clip [1, 96, 56, 56] 1204224
Conv [1, 96, 56, 56] 1204224
Conv [24, 96, 1, 1] 9216
Conv [24] 96
Conv [1, 24, 56, 56] 301056
Conv [144, 24, 1, 1] 13824
Conv [144] 576
Clip [1, 144, 56, 56] 1806336
Conv [1, 144, 56, 56] 1806336
Conv [144, 1, 3, 3] 5184
Conv [144] 576
Clip [1, 144, 56, 56] 1806336
Conv [1, 144, 56, 56] 1806336
Conv [24, 144, 1, 1] 13824
Conv [24] 96
Add [1, 24, 56, 56] 301056
Add [1, 24, 56, 56] 301056
Conv [1, 24, 56, 56] 301056
Conv [144, 24, 1, 1] 13824
Conv [144] 576
Clip [1, 144, 56, 56] 1806336
Conv [1, 

True

## 2-2-3. activation memory storage requirement

- Assuming the activations are stored to local memory and reuse multiple times, how much local memory storage is required to keep the activations?

In [3]:
import onnx
from onnx import shape_inference
from os import path
import sys
from tabulate import tabulate
from onnx import onnx_ml_pb2 as xpb2


onnx_model = onnx.load("./mobilenetv2-10.onnx", load_external_data=False)
onnx.checker.check_model(onnx_model)

inferred_model = shape_inference.infer_shapes(onnx_model)
print('shape inference complete ...')

def _parse_element(elem: xpb2.ValueInfoProto):
    name = getattr(elem, 'name', "None")
    data_type = "NA"
    shape_str = "NA"
    etype = getattr(elem, 'type', False)
    # print(elem.name)
    if etype:
        ttype = getattr(etype, 'tensor_type', False)
        if ttype:
            data_type = getattr(ttype, 'elem_type', 0)
            shape = getattr(elem.type.tensor_type, "shape", False)
            # print(name,shape)
            if shape:
                shape_str = "["
                dims = getattr(shape, 'dim', [])
                # print(dims)
                for dim in dims:
                    vals = getattr(dim,'dim_param',"?")
                    if vals: vals=1
                    else:vals = getattr(dim, 'dim_value', "?")
                    # print(vals)
                    if vals == 0: vals=1
                    shape_str += (str(vals) + ",")
                shape_str = shape_str.rstrip(",")
                shape_str += "]"
                print(shape_str)
    return name, data_type, shape_str

def get_valueproto_or_tensorproto_by_name(name: str, graph: xpb2.GraphProto):
    for i, node in enumerate(inferred_model.graph.node):
            if node.name == "":
                inferred_model.graph.node[i].name = str(i)
    input_nlist = [k.name for k in graph.input]
    initializer_nlist = [k.name for k in graph.initializer]
    value_info_nlist = [k.name for k in graph.value_info]
    output_nlist = [k.name for k in graph.output]

    # get tensor data
    if name in input_nlist:
        idx = input_nlist.index(name)
        return graph.input[idx], int(1)
    elif name in value_info_nlist:
        idx = value_info_nlist.index(name)
        return graph.value_info[idx], int(2)
    elif name in initializer_nlist:
        idx = initializer_nlist.index(name)
        return graph.initializer[idx], int(3)
    elif name in output_nlist:
        idx = output_nlist.index(name)
        return graph.output[idx], int(4)
    else:
        print("[ERROR MASSAGE] Can't find the tensor: ", name)
        print('input_nlist:\n', input_nlist)
        print('===================')
        print('value_info_nlist:\n', value_info_nlist)
        print('===================')
        print('initializer_nlist:\n', initializer_nlist)
        print('===================')
        print('output_nlist:\n', output_nlist)
        print('===================')
        return False, 0

def cal_tensor_mem_size(elem_type: str, shape: [int]):
    """ given the element type of the tensor and its shape, and return its memory size.

    Utility.

    Args:
        ttype: the type of the element of the given tensor. format: 'int', ...
        shape: the shape of the given tensor. format: [] of int

    Returns:
        mem_size: int
    """
    # init
    mem_size = int(1)
    # traverse the list to get the number of the elements
    for num in shape:
        mem_size *= num
    # multiple the size of variable with the number of the elements
    # "FLOAT": 1,
    # "UINT8": 2,
    # "INT8": 3,
    # "UINT16": 4,
    # "INT16": 5,
    # "INT32": 6,
    # "INT64": 7,
    # # "STRING" : 8,
    # "BOOL": 9,
    # "FLOAT16": 10,
    # "DOUBLE": 11,
    # "UINT32": 12,
    # "UINT64": 13,
    # "COMPLEX64": 14,
    # "COMPLEX128": 15
    if elem_type == 1:
        mem_size *= 4
    elif elem_type == 2:
        mem_size *= 1
    elif elem_type == 3:
        mem_size *= 1
    elif elem_type == 4:
        mem_size *= 2
    elif elem_type == 5:
        mem_size *= 2
    elif elem_type == 6:
        mem_size *= 4
    elif elem_type == 7:
        mem_size *= 8
    elif elem_type == 9:
        mem_size *= 1
    elif elem_type == 10:
        mem_size *= 2
    elif elem_type == 11:
        mem_size *= 8
    elif elem_type == 12:
        mem_size *= 4
    elif elem_type == 13:
        mem_size *= 8
    elif elem_type == 14:
        mem_size *= 8
    elif elem_type == 15:
        mem_size *= 16
    else:
        print("Undefined data type")

    return mem_size



def get_bandwidth(graph: xpb2.GraphProto):
    try:
        Activation_list = []
        total_activation = 0
        unknown_tensor_list = []
        # traverse all the nodes
        for nodeProto in graph.node:
            # init variables
            # print(nodeProto)
            read_mem_BW_each_layer = 0
            write_mem_BW_each_layer = 0
            total_each_layer = 0
            # traverse all input tensor
            for input_name in nodeProto.input:
                # get the TensorProto/ValueInfoProto by searching its name
                proto, type_Num = get_valueproto_or_tensorproto_by_name(
                    input_name, graph)
                # if nodeProto.op_type == "Gemm": print(type_Num)
                # print(proto,type_Num)
                # parse the ValueInfoProto/TensorProto
                if proto:
                    if type_Num == 3:
                        dtype = getattr(proto, 'data_type', False)
                        # get the shape of the tensor
                        shape = getattr(proto, 'dims', [])
                        # print(shape_str)
                        # print(proto.name,shape)
                        
                    elif type_Num == 1 or type_Num == 2:
                        name, dtype, shape_str = _parse_element(proto)
                        # print(proto.name,shape_str)
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        # print(name,shape_str)
                        shape = []
                        # print(f"3 = {dtype}, shape={shape}")
                        for dim in shape_str:
                            if len(dim) > 0:
                                shape.append(int(dim))
                    else:
                        print(
                            '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                            input_name, ' is from a wrong list !')
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        input_name, ' is no found !')
                    unknown_tensor_list.append(
                        (nodeProto.name, input_name, nodeProto.op_type))
                # calculate the tensor size in btye
                # print(nodeProto)
                # if(len(shape)<3) and type_Num==3 :continue
                read_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)
                print(type_Num,nodeProto.name,shape,cal_tensor_mem_size(dtype, shape),read_mem_BW_each_layer)
            
            total_each_layer = read_mem_BW_each_layer

            # store into tuple
            temp_tuple = (nodeProto.name, total_each_layer)
            #append it
            Activation_list.append(temp_tuple)
            # accmulate the value
            total_activation += total_each_layer

        # display the mem_bw of eahc layer
        columns = ['layer','each_layer']
        # resort the list
        Activation_list = sorted(Activation_list,
                             key=lambda Layer: Layer[1],
                             reverse=True)
        print(tabulate(Activation_list, headers=columns))
        print(
            '====================================================================================\n'
        )
        # display it
        print(
            "The activation memory for the model is: \n",
            total_activation, '(bytes)\n',
            float(total_activation) / float(1000000), '(MB)\n')
        # display the unknown tensor
        columns = ['op_name', 'unfound_tensor', 'op_type']
        print(tabulate(unknown_tensor_list, headers=columns))
        print(
            '====================================================================================\n'
        )
    except Exception as e:
        print("[ERROR MASSAGE] Unable to display: " + str(e))
        return False

    return True

#從這裡開始
print("start")
get_bandwidth(inferred_model.graph)


shape inference complete ...
start
[1,3,224,224]
1 Conv_0 [1, 3, 224, 224] 602112 602112
3 Conv_0 [32, 3, 3, 3] 3456 605568
3 Conv_0 [32] 128 605696
[1,32,112,112]
2 Clip_1 [1, 32, 112, 112] 1605632 1605632
[1,32,112,112]
2 Conv_2 [1, 32, 112, 112] 1605632 1605632
3 Conv_2 [32, 1, 3, 3] 1152 1606784
3 Conv_2 [32] 128 1606912
[1,32,112,112]
2 Clip_3 [1, 32, 112, 112] 1605632 1605632
[1,32,112,112]
2 Conv_4 [1, 32, 112, 112] 1605632 1605632
3 Conv_4 [16, 32, 1, 1] 2048 1607680
3 Conv_4 [16] 64 1607744
[1,16,112,112]
2 Conv_5 [1, 16, 112, 112] 802816 802816
3 Conv_5 [96, 16, 1, 1] 6144 808960
3 Conv_5 [96] 384 809344
[1,96,112,112]
2 Clip_6 [1, 96, 112, 112] 4816896 4816896
[1,96,112,112]
2 Conv_7 [1, 96, 112, 112] 4816896 4816896
3 Conv_7 [96, 1, 3, 3] 3456 4820352
3 Conv_7 [96] 384 4820736
[1,96,56,56]
2 Clip_8 [1, 96, 56, 56] 1204224 1204224
[1,96,56,56]
2 Conv_9 [1, 96, 56, 56] 1204224 1204224
3 Conv_9 [24, 96, 1, 1] 9216 1213440
3 Conv_9 [24] 96 1213536
[1,24,56,56]
2 Conv_10 [1, 24,

True

In [27]:
import onnx
from onnx import shape_inference
from os import path
import sys
from tabulate import tabulate
from onnx import onnx_ml_pb2 as xpb2


onnx_model = onnx.load("./mobilenetv2-10.onnx", load_external_data=False)
onnx.checker.check_model(onnx_model)

inferred_model = shape_inference.infer_shapes(onnx_model)
print('shape inference complete ...')

def _parse_element(elem: xpb2.ValueInfoProto):
    name = getattr(elem, 'name', "None")
    data_type = "NA"
    shape_str = "NA"
    etype = getattr(elem, 'type', False)
    # print(elem.name)
    if etype:
        ttype = getattr(etype, 'tensor_type', False)
        if ttype:
            data_type = getattr(ttype, 'elem_type', 0)
            shape = getattr(elem.type.tensor_type, "shape", False)
            # print(name,shape)
            if shape:
                shape_str = "["
                dims = getattr(shape, 'dim', [])
                # print(dims)
                for dim in dims:
                    vals = getattr(dim,'dim_param',"?")
                    if vals: vals=1
                    else:vals = getattr(dim, 'dim_value', "?")
                    # print(vals)
                    if vals == 0: vals=1
                    shape_str += (str(vals) + ",")
                shape_str = shape_str.rstrip(",")
                shape_str += "]"
                # print(shape_str)
    return name, data_type, shape_str

def get_valueproto_or_tensorproto_by_name(name: str, graph: xpb2.GraphProto):
    for i, node in enumerate(inferred_model.graph.node):
            if node.name == "":
                inferred_model.graph.node[i].name = str(i)
    input_nlist = [k.name for k in graph.input]
    initializer_nlist = [k.name for k in graph.initializer]
    value_info_nlist = [k.name for k in graph.value_info]
    output_nlist = [k.name for k in graph.output]

    # get tensor data
    if name in input_nlist:
        idx = input_nlist.index(name)
        return graph.input[idx], int(1)
    elif name in value_info_nlist:
        idx = value_info_nlist.index(name)
        return graph.value_info[idx], int(2)
    elif name in initializer_nlist:
        idx = initializer_nlist.index(name)
        return graph.initializer[idx], int(3)
    elif name in output_nlist:
        idx = output_nlist.index(name)
        return graph.output[idx], int(4)
    else:
        print("[ERROR MASSAGE] Can't find the tensor: ", name)
        print('input_nlist:\n', input_nlist)
        print('===================')
        print('value_info_nlist:\n', value_info_nlist)
        print('===================')
        print('initializer_nlist:\n', initializer_nlist)
        print('===================')
        print('output_nlist:\n', output_nlist)
        print('===================')
        return False, 0

def cal_tensor_mem_size(elem_type: str, shape: [int]):
    """ given the element type of the tensor and its shape, and return its memory size.

    Utility.

    Args:
        ttype: the type of the element of the given tensor. format: 'int', ...
        shape: the shape of the given tensor. format: [] of int

    Returns:
        mem_size: int
    """
    # init
    mem_size = int(1)
    # traverse the list to get the number of the elements
    for num in shape:
        mem_size *= num
    # multiple the size of variable with the number of the elements
    # "FLOAT": 1,
    # "UINT8": 2,
    # "INT8": 3,
    # "UINT16": 4,
    # "INT16": 5,
    # "INT32": 6,
    # "INT64": 7,
    # # "STRING" : 8,
    # "BOOL": 9,
    # "FLOAT16": 10,
    # "DOUBLE": 11,
    # "UINT32": 12,
    # "UINT64": 13,
    # "COMPLEX64": 14,
    # "COMPLEX128": 15
    if elem_type == 1:
        mem_size *= 4
    elif elem_type == 2:
        mem_size *= 1
    elif elem_type == 3:
        mem_size *= 1
    elif elem_type == 4:
        mem_size *= 2
    elif elem_type == 5:
        mem_size *= 2
    elif elem_type == 6:
        mem_size *= 4
    elif elem_type == 7:
        mem_size *= 8
    elif elem_type == 9:
        mem_size *= 1
    elif elem_type == 10:
        mem_size *= 2
    elif elem_type == 11:
        mem_size *= 8
    elif elem_type == 12:
        mem_size *= 4
    elif elem_type == 13:
        mem_size *= 8
    elif elem_type == 14:
        mem_size *= 8
    elif elem_type == 15:
        mem_size *= 16
    else:
        print("Undefined data type")

    return mem_size
    
def get_activation_memory(graph: xpb2.GraphProto):

    All_tensor_size = []
    Total_size =0 
    
    node_nList = [k.name for k in graph.node]
    input_nlist = [k.name for k in graph.input]
    initializer_nlist = [k.name for k in graph.initializer]
    value_info_nlist = [k.name for k in graph.value_info]
    output_nlist = [k.name for k in graph.output]
    idx_list = {}
    for node in graph.node:
        if node.op_type in idx_list:
            idx_list[node.op_type][node.name] = node_nList.index(node.name)
        else:
            idx_list[node.op_type] = {
                node.name: node_nList.index(node.name)
            }
    All_type = idx_list.keys()
    for ty in All_type:
        for idx in idx_list[ty].values():
            # print(ty,idx)
            list_of_data_num = []

            # get input tensor proto
            # for input_name in graph.node[idx].input:
            #     # print(input_name)
                
            #     # get tensor data
            #     if input_name in input_nlist:
            #         name_idx = input_nlist.index(input_name)
            #         data = graph.input[name_idx]
            #         type_num = int(1)
            #     elif input_name in value_info_nlist:
            #         name_idx = value_info_nlist.index(input_name)
            #         data = graph.value_info[name_idx]
            #         type_num = int(2)
            #     elif input_name in initializer_nlist:
            #         name_idx = initializer_nlist.index(input_name)
            #         data = graph.initializer[name_idx]
            #         type_num = int(3)
            #     elif input_name in output_nlist:
            #         name_idx = output_nlist.index(input_name)
            #         data = graph.output[name_idx]
            #         type_num = int(4)
            #     else:
            #         print("Can't find the tensor: ", input_name)
            #         print('input_nlist:\n', input_nlist)
            #         print('===================')
            #         print('value_info_nlist:\n', value_info_nlist)
            #         print('===================')
            #         print('initializer_nlist:\n', initializer_nlist)
            #         print('===================')
            #         print('output_nlist:\n', output_nlist)
            #         print('===================')
            
            #     list_of_data_num.append((data, type_num))
                # print(len(list_of_data_num))
            list_temp = [
                    graph.node[idx].name,
                ]
            if graph.node[idx].output[0] in input_nlist:
                name_idx = input_nlist.index(graph.node[idx].output[0])
                data = graph.input[name_idx]
                type_num = int(1)
            elif graph.node[idx].output[0] in value_info_nlist:
                name_idx = value_info_nlist.index(graph.node[idx].output[0])
                data = graph.value_info[name_idx]
                type_num = int(2)
            elif graph.node[idx].output[0] in initializer_nlist:
                name_idx = initializer_nlist.index(graph.node[idx].output[0])
                data = graph.initializer[name_idx]
                type_num = int(3)
            elif graph.node[idx].output[0] in output_nlist:
                name_idx = output_nlist.index(graph.node[idx].output[0])
                data = graph.output[name_idx]
                type_num = int(4)
            else:
                print("Can't find the tensor: ", graph.node[idx].output[0])
                print('input_nlist:\n', input_nlist)
                print('===================')
                print('value_info_nlist:\n', value_info_nlist)
                print('===================')
                print('initializer_nlist:\n', initializer_nlist)
                print('===================')
                print('output_nlist:\n', output_nlist)
                print('===================')
            # list_of_data_num.append((data, type_num))
            # shape = getattr(proto, 'dims', [])
            # print(graph.node[idx].name,_parse_element(data))
            name, data_type, shape_str = _parse_element(data)
            shape_str = shape_str.strip('[]')
            shape_str = shape_str.split(',')
            shape = []
            for dim in shape_str:
                if not dim: continue
                else:shape.append(int(dim))
            mem_size = cal_tensor_mem_size(data_type, shape)
            Total_size += mem_size
            list_temp.append(mem_size)
            # print(shape,list_temp)
            All_tensor_size.append(list_temp)
    columns = [
            'Conv_name', 
            'Output_tensor'
        ]

    # resort the list
    All_tensor_size = sorted(All_tensor_size,
                                  key=lambda Layer: Layer[-1],
                                  reverse=True)
    print('list_of_layer RESORTED ! ')
    print(tabulate(All_tensor_size, headers=columns))
    print(f"Total memory size = {Total_size}")

        
# name, dtype, shape_str = _parse_element(proto)
# dtype = getattr(proto, 'data_type', False)
get_activation_memory(inferred_model.graph)

shape inference complete ...
list_of_layer RESORTED ! 
Conv_name               Output_tensor
--------------------  ---------------
Conv_5                        4816896
Clip_6                        4816896
Conv_10                       1806336
Conv_12                       1806336
Conv_16                       1806336
Clip_11                       1806336
Clip_13                       1806336
Clip_17                       1806336
Conv_0                        1605632
Conv_2                        1605632
Clip_1                        1605632
Clip_3                        1605632
Conv_7                        1204224
Clip_8                        1204224
Conv_4                         802816
Conv_21                        602112
Conv_23                        602112
Conv_27                        602112
Conv_29                        602112
Conv_33                        602112
Clip_22                        602112
Clip_24                        602112
Clip_28                        60

In [ ]:
#test
import onnx
from onnx import helper, numpy_helper

# 创建 Gemm 节点
gemm_node = helper.make_node(
    'Gemm',  # 节点类型
    ['A', 'B', 'C'],  # 输入
    ['Y'],  # 输出
    alpha=1.0,  # alpha 参数
    beta=1.0,  # beta 参数
    transA=0,  # transA 参数
    transB=0   # transB 参数
)

# 创建图
graph = helper.make_graph(
    [gemm_node],
    "gemm_example",
    [
        helper.make_tensor_value_info('A', onnx.TensorProto.FLOAT, [3, 4]),
        helper.make_tensor_value_info('B', onnx.TensorProto.FLOAT, [4, 5]),
        helper.make_tensor_value_info('C', onnx.TensorProto.FLOAT, [3, 5])
    ],
    [helper.make_tensor_value_info('Y', onnx.TensorProto.FLOAT, [3, 5])]
)

# 创建模型
model = helper.make_model(graph)

# 保存模型
onnx.save(model, "gemm_example.onnx")